In [2]:
import numpy as np
import cv2

## The Summary of MobileNet

In [12]:
def mobilenet_summary(net, layer_type=False, layer_shape=False, 
                      get_flops=False, get_memory=False,
                      get_total_time=False, get_layer_time=False):
    
    # check if the model is empty
    if net.empty():
        print("Net is empty")
        return
        
    # get the types of layers used in the model
    if layer_type:
        print("\nNet contains:")
        for t in net.getLayerTypes():
            print("\t{} layers of type {}".format(net.getLayersCount(t), t))
        
    
    
    # get the tensor shapes for the loaded model and specified input shape
    if layer_shape:
        layers_ids, in_shapes, out_shapes = net.getLayersShapes([1, 3, 224, 224])
        layers_names = net.getLayerNames()
        print("\nNet layers shapes: ")
        for l in range(len(layers_names)):
            in_num, out_num = len(in_shapes[l]), len(out_shapes[l])
            print("  Layer {} has {} inputs and {} outputs".format(layers_names[l],
                                                                 in_num, out_num))
            for i in range(in_num):
                print("\tinput #{} has shape {}".format(i, in_shapes[l][i].flatten()))
            for i in range(out_num):
                print("\toutput #{} has shape {}".format(i, out_shapes[l][i].flatten()))
            
    # compute the number of FLOPs
    if get_flops:
        print("\ngflops: ", net.getFLOPS((1, 3, 224, 224)) * 1e-9)
    
    # report the amount of memory consumed for storing weights and intermediate tensors
    if get_memory:
        w, b = net.getMemoryConsumption((1, 3, 224, 224))
        print("\nweights (mb):", w * 1e-6, ", blobs (mb):", b * 1e-6)
    
    # perform a forward pass for a mock input
    blob = cv2.dnn.blobFromImage(np.zeros((224, 224, 3), np.uint8), 1, (224, 224))
    net.setInput(blob)
    net.forward()
    
    # report the total time
    if get_total_time:
        total, timings = net.getPerfProfile()
        tick2ms = 1000 / cv2.getTickFrequency()
        print("\ninference (ms): {:2f}".format(total * tick2ms))
    
    # report the per layer inference time
    if get_layer_time:
        layer_names = net.getLayerNames()
        print("\n{: <30} {}".format("LAYER", "TIME (ms)"))
        for (i,t) in enumerate(timings):
            print("{: <30} {:.2f}".format(layer_names[i], t[0] * tick2ms))

In [13]:
model_path = "./model/mobilenet_v2.caffemodel"
proto_path = "./model/mobilenet_v2_deploy.prototxt"
mobilenet = cv2.dnn.readNetFromCaffe(proto_path, model_path)

In [14]:
mobilenet_summary(mobilenet, layer_type=True)


Net contains:
	53 layers of type BatchNorm
	54 layers of type Convolution
	10 layers of type Eltwise
	1 layers of type Pooling
	36 layers of type ReLU
	53 layers of type Scale
	1 layers of type Softmax
	1 layers of type __NetInputLayer__


In [15]:
mobilenet_summary(mobilenet, layer_shape=True)


Net layers shapes: 
  Layer conv1 has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [  1   3 224 224]
  Layer conv1/bn has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [  1  32 112 112]
  Layer conv1/scale has 1 inputs and 1 outputs
	input #0 has shape [  1  32 112 112]
	output #0 has shape [  1  32 112 112]
  Layer relu1 has 1 inputs and 1 outputs
	input #0 has shape [  1  32 112 112]
	output #0 has shape [  1  32 112 112]
  Layer conv2_1/expand has 1 inputs and 1 outputs
	input #0 has shape [  1  32 112 112]
	output #0 has shape [  1  32 112 112]
  Layer conv2_1/expand/bn has 1 inputs and 1 outputs
	input #0 has shape [  1  32 112 112]
	output #0 has shape [  1  32 112 112]
  Layer conv2_1/expand/scale has 1 inputs and 1 outputs
	input #0 has shape [  1  32 112 112]
	output #0 has shape [  1  32 112 112]
  Layer relu2_1/expand has 1 inputs and 1 outputs
	input #0 has shape [  1  32 112 112]
	output #0 has shape

In [17]:
mobilenet_summary(mobilenet, get_flops=True, get_memory=True)


gflops:  16.475390984

weights (mb): 14.160756 , blobs (mb): 141.816768


In [18]:
mobilenet_summary(mobilenet, get_total_time=True, get_layer_time=True)


inference (ms): 458.311762

LAYER                          TIME (ms)
conv1                          11.17
conv1/bn                       0.00
conv1/scale                    0.00
relu1                          0.00
conv2_1/expand                 21.40
conv2_1/expand/bn              0.00
conv2_1/expand/scale           0.00
relu2_1/expand                 0.00
conv2_1/dwise                  56.56
conv2_1/dwise/bn               0.00
conv2_1/dwise/scale            0.00
relu2_1/dwise                  0.00
conv2_1/linear                 2.24
conv2_1/linear/bn              0.00
conv2_1/linear/scale           0.00
conv2_2/expand                 5.90
conv2_2/expand/bn              0.00
conv2_2/expand/scale           0.00
relu2_2/expand                 0.00
conv2_2/dwise                  50.19
conv2_2/dwise/bn               0.00
conv2_2/dwise/scale            0.00
relu2_2/dwise                  0.00
conv2_2/linear                 1.19
conv2_2/linear/bn              0.00
conv2_2/linear/scale      